# Finding closest point on given line of longitude
#### Given longitude and latitude of a point P=($\theta_1$, $\lambda_1$) and a latitude $\lambda_2$, let's find longitude $\theta_2$ such that, great-circle distance $d$ (distance along the surface of a sphere) is the smallest.

#### Let's simplify this problem: given two points A=($\theta_A$, $\lambda_A$), B=($\theta_B$, $\lambda_B$) find their great-circle distance $d$
This [wikipedia article](https://en.wikipedia.org/wiki/Great-circle_distance) let's us explore this topic.
All we have to do is to find the central angle $\Delta\sigma$ (the angle between OA and OC, where O is the sphere's center). Note that $\frac{d}{2\pi R} = \frac{\Delta\sigma}{2\pi}$, which concludes the search of d.
Now using the formula
$\Delta\sigma =
2\arcsin \sqrt{ \sin^{2}\frac{\theta_B-\theta_A}{2} +
(1 - \sin^{2}\frac{\theta_B-\theta_A}{2}
\sin^{2}\frac{\theta_A+\theta_B}{2}) \cdot
\sin^{2}\frac{\lambda_B-\lambda_A}{2}}$
We are able to calculate $\Delta\sigma$ thus calculating $d$.

![sphere.png](gcd_sphere.png "graet-circle distance")

#### Now that we can calculate the great-circle distance, let's come back to the original problem.

Let's introduce some substitutions:
* $x=\Phi_2$
* $A=\Phi_1$
* $B=\sin^{2}\frac{\lambda_2-\lambda_1}{2}$
Now the equation looks like this:
$\Delta\sigma = 
2\arcsin \sqrt{ \sin^{2}\frac{x-A}{2} +
(1 - \sin^{2}\frac{x-A}{2}
     \sin^{2}\frac{x+A}{2}) \cdot
     B$
     
Let's calculate the first derivative
$\frac{2}{\sqrt{1-t}} \cdot
\frac{1}{2\sqrt{t}} \cdot
(\frac{\sin x-A}{2} - \frac{B\sin x+A}{2} - \frac{B\sin x-A}{2})$
where $t=
\sin^{2}\frac{x-A}{2} +
(1 - \sin^{2}\frac{x-A}{2}
\sin^{2}\frac{x+A}{2}) \cdot
B$

The function reaches extreme when its derivative is equal 0
$\sin(x-A) - B(\sin(x+A) + \sin(x-A)) = 0$

Using some simple transformations we get
$x = \arctan\frac{\tan A}{1-2B} + k\pi$

And finally
$\Phi_2 = \arctan{\frac{\tan \Phi_1}{1-2sin^{2}\frac{\lambda_2-\lambda_1}{2}} + k\pi$

    ### Checkout [Desmos graph]('https://www.desmos.com/calculator/2iyi19yuiv')
![gcd_graph.png](gcd_graph.png "graet-circle distance")

In [1]:
import numpy as np

In [2]:
# this is a different version of the same formula
def dist(long1, lat1, long2, lat2):
    """
        a = sin²(Δϕ/2) + cos(ϕ1)cos(ϕ2)sin²(Δλ/2)
        d = 2R*arctan2(sqrt(a),sqrt(1−a))
    """
    long1, lat1 = np.deg2rad(long1), np.deg2rad(lat1),
    long2, lat2 = np.deg2rad(long2), np.deg2rad(lat2),
    R = 1
    a = np.sin((lat2 - lat1) / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin((long2 - long1) / 2) ** 2
    d = (2 * R * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    return d

In [3]:
def calc_extremes(long1, lat1, long2):
    long1r, lat1r, long2r = np.deg2rad([long1, lat1, long2])
    lat2 = np.arctan(np.tan(lat1r) / (1 - 2 * (np.sin((long2r - long1r) / 2) ** 2))) * 180 / np.pi

    # since we are considering only a semicircle, the other extreme will be at one of the poles
    # if we were to consider the whole circle, then the other point would be at lat2+pi
    # with distance equal 2piR/2-dist
    candidate_points = [(long2, lat2), (long2, -90), (long2, 90)]
    distances = [dist(long1, lat1, *p) for p in candidate_points]
    points_distances = list(zip(candidate_points, distances))

    return (min(points_distances, key=lambda x: x[1]),
            max(points_distances, key=lambda x: x[1]))

In [4]:
LONG1, LAT1 = 0, 20.0
LONG2 = 45.0

extremes = calc_extremes(LONG1, LAT1, LONG2)
print(f'Minimum = {extremes[0][1]} at {extremes[0][0]}')
print(f'Maximum = {extremes[1][1]} at {extremes[1][0]}')

Minimum = 0.7267750543196971 at (45.0, 27.236313475170725)
Maximum = 1.9198621771937625 at (45.0, -90)
